<h1 align=center><font size = 5>Capstone Project - The Battle of Neighborhoods</font></h1>

# Housing Sales Prices & Venues Data Analysis of Bangkok 
## Introduction
### A.1. Description & Disscusion of the Background 
Bangkok is the capital and most populous city of Thailand. It is known in Thai as Krung Thep Maha Nakhon or simply Krung Thep. The city occupies 1,568.7 square kilometres (605.7 sq mi) in the Chao Phraya River delta in central Thailand, and has a population of over eight million, or 12.6 percent of the country's population. Over fourteen million people (22.2 percent) lived within the surrounding Bangkok Metropolitan Region at the 2010 census, making Bangkok the nation's primate city, significantly dwarfing Thailand's other urban centres in both size and importance to the national economy .The city of Bangkok has a population of 8,305,218 according to the 2010 census, or 12.6 percent of the national population. In 2018, the population has been estimated to be about 10 million. The city is divided into 50 districts in to total.Bangkok city's population density is an average 5,300 per square kilometer. When we think of it by the investor,we expect from them to prefer the districts where there is a lower real estate cost and the type of business they want to install is less intense because Bangkok is the economic centre of Thailand, and the heart of the country's investment and development.However, it is difficult to obtain information that will guide investors in direction. When we consider all these problems, we can create a map and information chart where the real estate index is placed on Bangkok and each district is clustered according to the venue density.


### A.2. Data Desciption
To consider the porblem we can list the datas as below:
We found  geographic coordinate of Bangkok in website https://www.geonames.org/ and convert degrees minutes secondesd to decimal degrees. We found Housing Sales Prices in website https://www.treasury.go.th/th/summary-of-land-valuation/ collected latest per square meter Housing Sales Price (HSP) Averages for each Borough of Bangkok housing retail. Ceaned the data and reduced it to city of bangkok where we used it to create choropleth map of Housing sales Price Index of Bangkok.
I used Forsquare API to get the most common venues of given Borough of Bangkok.There are not too many public datas related to demographic and social parameters for the city of Bangkok.you must set-up your own data tables in most cases and used Google Map, ‘Search Nearby’ option to get the center coordinates of the each Borough.




In [1]:
# Import libraries
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import pandas as pd

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup

# Import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Data Research and Preparation
## Import the bangkok district Data 
### Arrondissements or Bangkok  CSV (administrative districts)
Bangkok  is divided into 50 Arrondissements and initially looking to get this data by scraping the relevent Wikipedia page (https://en.wikipedia.org/wiki/Bangkok), fortunately, after much research, this data is available on the web and can be manipulated and cleansed to provide a meaningful dataset to use.



In [2]:
# # Download the dataset and read it into a pandas dataframe.
df_bangkok = pd.read_csv('https://raw.githubusercontent.com/wittawut-vita/bangkok-Housing-Sale-Prices/master/data%20file.csv')
print('Dataset download and read into a pandas dataframe!')

Dataset download and read into a pandas dataframe!


In [3]:
df_bangkok.head()

,num,district,province,latitude,longitude,housing_prices
0,1,Bangkok Yai,Bangkok,13.737222,100.474167,9836
1,2,Bangkok Noi,Bangkok,13.765278,100.469722,6557
2,3,Bang Khae,Bangkok,13.719167,100.392778,4918
3,4,Bang Sue,Bangkok,13.819167,100.527778,3934
4,5,Bang Khen,Bangkok,13.868889,100.627500,26230


In [4]:
df_bangkok.shape

(50, 6)

# Methodology
As a database, I used GitHub repository in my study. Let's download and import the data on police department incidents using *pandas* `read_csv()` method.
Now that we reduced the data a little bit, let's visualize (used python folium library ) where these Housing Sales Prices took place in the city of Bankgkok. We will use the default style and we will initialize the zoom level to 12.
Now let's superimpose the locations of the Housing sale prices onto the map. The way to do that in Folium is to create a feature group with its own features and style and then add it to the bangkok_map.


In [5]:
# Bangkok  latitude and longitude values
latitude = 13.7525
longitude = 100.494167

In [6]:
#Create map and display it
bangkok_map = folium.Map(location=[latitude,longitude],zoom_start=12,)

# add the District  as blue circle markers
for lat, lng, label in zip(df_bangkok.latitude,df_bangkok.longitude, df_bangkok.district):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(bangkok_map)



bangkok_map
   
       

# Use the Foursquare API to explore the Arrondissements of Paris (Neighborhoods)
 We utilized the Foursquare API to explore the boroughs and segment them. We designed the limit as 100 venue and the radius 750 meter for each borough from given latitude and longitude inforations.

In [7]:
CLIENT_ID = 'OWON42M4PI5W4TABXIYOETS3AGAQ1CACDDZ23FDQ0YJWO1WM' # your Foursquare ID
CLIENT_SECRET = 'XSMMVALJIMXPVRBY4MSOSCVYQSVLGNMZMLCZMO4Q5BBP5GRJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OWON42M4PI5W4TABXIYOETS3AGAQ1CACDDZ23FDQ0YJWO1WM
CLIENT_SECRET:XSMMVALJIMXPVRBY4MSOSCVYQSVLGNMZMLCZMO4Q5BBP5GRJ


# Let's explore the first neighborhood in our dataframe
Get the neighborhood's name.

In [8]:
# Get the Neighborhood's bangkok name.
df_bangkok.loc[0,'district']

'Bangkok Yai'

### Get the neighborhood's latitude and longitude values.

In [9]:
neighborhood_latitude = df_bangkok.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = df_bangkok.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = df_bangkok.loc[0, 'district'] # neighborhood name

print('latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

latitude and longitude values of Bangkok Yai are 13.737222000000001, 100.47416700000001.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 750 meters.

In [10]:
# First, let's create the GET request URL. Name your URL url.
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 750 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OWON42M4PI5W4TABXIYOETS3AGAQ1CACDDZ23FDQ0YJWO1WM&client_secret=XSMMVALJIMXPVRBY4MSOSCVYQSVLGNMZMLCZMO4Q5BBP5GRJ&v=20180605&ll=13.737222000000001,100.47416700000001&radius=750&limit=100'

In [11]:
# Send the GET request and examine the resutls
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e4118836001fe001b1cb2f2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Wat Tha Phra',
  'headerFullLocation': 'Wat Tha Phra, Bangkok',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 17,
  'suggestedBounds': {'ne': {'lat': 13.743972006750008,
    'lng': 100.48110280195594},
   'sw': {'lat': 13.730471993249994, 'lng': 100.46723119804408}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bdd3232645e0f47a1156b19',
       'name': 'KFC (เคเอฟซี)',
       'location': {'address': 'Charan Sanit Wong Rd',
        'crossStreet': 'in ESSO',
        'lat': 13.74142853678466,
        'lng': 100.47078988005451,
        'labeledLatLngs':

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Now we are ready to clean the json and structure it into a pandas dataframe.

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,KFC (เคเอฟซี),Fast Food Restaurant,13.741429,100.470790
1,ลิ่มซังข้าวมันไก่,Asian Restaurant,13.730746,100.473320
2,7-Eleven (เซเว่น อีเลฟเว่น),Convenience Store,13.739921,100.474249
3,7-Eleven (เซเว่น อีเลฟเว่น),Convenience Store,13.740172,100.470578
4,7-Eleven (เซเว่น อีเลฟเว่น),Convenience Store,13.737108,100.471113


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


## Explore Neighborhoods in Bangkok 

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called bangkok_venues.
bangkok_venues = getNearbyVenues(names=df_bangkok['district'],
                                   latitudes=df_bangkok['latitude'],
                                   longitudes=df_bangkok['longitude']
                                  )

Bangkok Yai
Bangkok Noi
Bang Khae
Bang Sue
Bang Khen
Bang Kapi
Don Mueang
Thon Buri
Taling Chan
Samphanthawong
Rat Burana
Phra Khanong
Phaya Thai
Phasi Charoen
Pathum Wan
Nong Khaem
Nong Chok
Min Buri
Lat Krabang
Khlong Toei
Khlong San
Dusit
Ratchathewi
Din Daeng
Saphan Sung
Bang Bon
Chom Thong
Thawi Watthana
Huai Khwang
Bueng Kum
Wang Thonglang
Bang Phlat
Lat Phrao
Khan Na Yao
Khlong Sam Wa
Lak Si
Chatuchak
Sai Mai
Thung Khru
Bang Na
Sathon
Prawet
Watthana
Suan Luang
Yan Nawa
Bang Rak
Bang Khun Thian
Pom Prap Sattru Phai
Phra Nakhon
Bang Kho Laem


###  Let's check the size of the resulting dataframe

In [17]:
print(bangkok_venues.shape)
bangkok_venues.head()

(1880, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bangkok Yai,13.737222,100.474167,KFC (เคเอฟซี),13.741429,100.470790,Fast Food Restaurant
1,Bangkok Yai,13.737222,100.474167,ลิ่มซังข้าวมันไก่,13.730746,100.473320,Asian Restaurant
2,Bangkok Yai,13.737222,100.474167,7-Eleven (เซเว่น อีเลฟเว่น),13.739921,100.474249,Convenience Store
3,Bangkok Yai,13.737222,100.474167,7-Eleven (เซเว่น อีเลฟเว่น),13.740172,100.470578,Convenience Store
4,Bangkok Yai,13.737222,100.474167,7-Eleven (เซเว่น อีเลฟเว่น),13.737108,100.471113,Convenience Store


### Let's check how many venues were returned for each neighborhood

In [18]:
bangkok_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bang Bon,7,7,7,7,7,7
Bang Kapi,53,53,53,53,53,53
Bang Khae,22,22,22,22,22,22
Bang Khen,11,11,11,11,11,11
Bang Kho Laem,42,42,42,42,42,42
Bang Khun Thian,5,5,5,5,5,5
Bang Na,19,19,19,19,19,19
Bang Phlat,17,17,17,17,17,17
Bang Rak,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(bangkok_venues['Venue Category'].unique())))

There are 208 uniques categories.


###  Analyze Each Neighborhood

In [20]:
# one hot encoding
bangkok_onehot = pd.get_dummies(bangkok_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangkok_onehot['Neighborhood'] = bangkok_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangkok_onehot.columns[-1]] + list(bangkok_onehot.columns[:-1])
bangkok_onehot = bangkok_onehot[fixed_columns]

bangkok_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,BBQ Joint,Badminton Court,...,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
bangkok_onehot.shape

(1880, 208)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
bangkok_grouped = bangkok_onehot.groupby('Neighborhood').mean().reset_index()
bangkok_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,BBQ Joint,...,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Bang Bon,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.000000,0.00000
1,Bang Kapi,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.018868,0.00000
2,Bang Khae,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
3,Bang Khen,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
4,Bang Kho Laem,0.000000,0.000000,0.000000,0.00,0.023810,0.000000,0.023810,0.000000,0.023810,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
5,Bang Khun Thian,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
6,Bang Na,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.105263,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
7,Bang Phlat,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
8,Bang Rak,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.00000
9,Bang Sue,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000


#### Let's confirm the new size

In [38]:
bangkok_grouped.shape

(50, 208)

#### Let's print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in bangkok_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bangkok_grouped[bangkok_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bang Bon----
              venue  freq
0   Thai Restaurant  0.14
1       Coffee Shop  0.14
2        Water Park  0.14
3  Asian Restaurant  0.14
4  Basketball Court  0.14


----Bang Kapi----
                  venue  freq
0       Thai Restaurant  0.09
1           Coffee Shop  0.09
2   Japanese Restaurant  0.08
3  Fast Food Restaurant  0.08
4     Convenience Store  0.06


----Bang Khae----
                venue  freq
0     Thai Restaurant  0.14
1        Noodle House  0.14
2        Dessert Shop  0.09
3   Convenience Store  0.09
4  Chinese Restaurant  0.09


----Bang Khen----
               venue  freq
0         Restaurant  0.18
1    Thai Restaurant  0.09
2            Dog Run  0.09
3  Fish & Chips Shop  0.09
4        Flea Market  0.09


----Bang Kho Laem----
                venue  freq
0   Convenience Store  0.14
1     Thai Restaurant  0.12
2  Chinese Restaurant  0.07
3         Coffee Shop  0.07
4        Noodle House  0.07


----Bang Khun Thian----
                venue  freq
0  Seafood 

             venue  freq
0  Thai Restaurant   0.2
1       Restaurant   0.2
2       Steakhouse   0.2
3      Coffee Shop   0.2
4       Hobby Shop   0.2


----Thawi Watthana----
              venue  freq
0   Thai Restaurant  0.33
1       Bus Station  0.17
2       Record Shop  0.17
3           Dog Run  0.17
4  Asian Restaurant  0.17


----Thon Buri----
              venue  freq
0      Noodle House  0.15
1   Thai Restaurant  0.10
2  Asian Restaurant  0.07
3      Dessert Shop  0.07
4     Train Station  0.05


----Thung Khru----
                   venue  freq
0      Hotpot Restaurant  0.18
1      Convenience Store  0.18
2  Vietnamese Restaurant  0.09
3          Grocery Store  0.09
4       Department Store  0.09


----Wang Thonglang----
              venue  freq
0       Coffee Shop  0.10
1      Noodle House  0.08
2   Thai Restaurant  0.05
3  Department Store  0.05
4  Asian Restaurant  0.05


----Watthana----
                 venue  freq
0                  Bar  0.10
1                 Café  0.08

## Let's put that into a pandas dataframe
### First, let's write a function to sort the venues in descending order.

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangkok_grouped['Neighborhood']

for ind in np.arange(bangkok_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangkok_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,Satay Restaurant,Fried Chicken Joint,Water Park,Asian Restaurant,Coffee Shop,Thai Restaurant,Basketball Court,Flea Market,Frozen Yogurt Shop,French Restaurant
1,Bang Kapi,Coffee Shop,Thai Restaurant,Fast Food Restaurant,Japanese Restaurant,Steakhouse,Convenience Store,Bakery,Noodle House,Som Tum Restaurant,Film Studio
2,Bang Khae,Noodle House,Thai Restaurant,Dessert Shop,Bar,Convenience Store,Chinese Restaurant,Bus Stop,Pharmacy,Farmers Market,Salad Place
3,Bang Khen,Restaurant,Dog Run,Coffee Shop,Noodle House,Food Truck,Boxing Gym,Gym Pool,Fish & Chips Shop,Thai Restaurant,Flea Market
4,Bang Kho Laem,Convenience Store,Thai Restaurant,Chinese Restaurant,Coffee Shop,Noodle House,Hotpot Restaurant,Badminton Court,Pub,Café,Smoothie Shop


#  Cluster Neighborhoods

Run k-means to cluster the neighborhood into 3 clusters.

In [42]:
# set number of clusters
kclusters = 3

bangkok_grouped_clustering =bangkok_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangkok_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 2, 1, 2, 0, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0,'Cluster_Labels',kmeans.labels_)

bangkok_merged = df_bangkok

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bangkok_merged = bangkok_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='district')

bangkok_merged.head() # check the last columns!

,num,district,province,latitude,longitude,housing_prices,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Bangkok Yai,Bangkok,13.737222,100.474167,9836,2,Convenience Store,Cosmetics Shop,Ice Cream Shop,BBQ Joint,Coffee Shop,Restaurant,Fast Food Restaurant,Steakhouse,Supermarket,Asian Restaurant
1,2,Bangkok Noi,Bangkok,13.765278,100.469722,6557,2,Noodle House,Convenience Store,Market,Food Truck,Som Tum Restaurant,Dessert Shop,Fast Food Restaurant,Thai Restaurant,Steakhouse,Coffee Shop
2,3,Bang Khae,Bangkok,13.719167,100.392778,4918,0,Noodle House,Thai Restaurant,Dessert Shop,Bar,Convenience Store,Chinese Restaurant,Bus Stop,Pharmacy,Farmers Market,Salad Place
3,4,Bang Sue,Bangkok,13.819167,100.527778,3934,2,Convenience Store,Thai Restaurant,Café,Pub,Night Market,Bar,Noodle House,Market,Train Station,Boutique
4,5,Bang Khen,Bangkok,13.868889,100.627500,26230,0,Restaurant,Dog Run,Coffee Shop,Noodle House,Food Truck,Boxing Gym,Gym Pool,Fish & Chips Shop,Thai Restaurant,Flea Market


Finally, let's visualize the resulting clusters

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi_1, cluster,poi in zip(bangkok_merged['latitude'], bangkok_merged['longitude'], bangkok_merged['district'], bangkok_merged['Cluster_Labels'],bangkok_merged['housing_prices']):
    label = folium.Popup(str(poi_1) + ' Cluster ' + str(cluster) + ' Housing Sales Prices '+ str(poi) + ' per square meter ', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster

#### Cluster 1

In [30]:
bangkok_merged.loc[bangkok_merged['Cluster_Labels'] == 0, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,district,housing_prices,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Bang Khae,4918,0,Noodle House,Thai Restaurant,Dessert Shop,Bar,Convenience Store,Chinese Restaurant,Bus Stop,Pharmacy,Farmers Market,Salad Place
4,Bang Khen,26230,0,Restaurant,Dog Run,Coffee Shop,Noodle House,Food Truck,Boxing Gym,Gym Pool,Fish & Chips Shop,Thai Restaurant,Flea Market
5,Bang Kapi,5574,0,Coffee Shop,Thai Restaurant,Fast Food Restaurant,Japanese Restaurant,Steakhouse,Convenience Store,Bakery,Noodle House,Som Tum Restaurant,Film Studio
6,Don Mueang,4590,0,Noodle House,Thai Restaurant,Coffee Shop,Convenience Store,Pizza Place,Asian Restaurant,Restaurant,Food & Drink Shop,Som Tum Restaurant,Halal Restaurant
7,Thon Buri,11475,0,Noodle House,Thai Restaurant,Asian Restaurant,Dessert Shop,Train Station,Farmers Market,Hotpot Restaurant,Convenience Store,Steakhouse,Bistro
8,Taling Chan,5902,0,Hobby Shop,Restaurant,Steakhouse,Thai Restaurant,Coffee Shop,Women's Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
9,Samphanthawong,22951,0,Noodle House,Chinese Restaurant,Coffee Shop,Seafood Restaurant,Café,Asian Restaurant,Dessert Shop,Thai Restaurant,Market,Tea Room
10,Rat Burana,4918,0,Coffee Shop,Convenience Store,Thai Restaurant,Noodle House,Chinese Restaurant,Ice Cream Shop,Bakery,Breakfast Spot,Flea Market,Steakhouse
12,Phaya Thai,13115,0,Coffee Shop,Thai Restaurant,Café,Noodle House,Som Tum Restaurant,Sushi Restaurant,Bar,Japanese Restaurant,Bakery,Restaurant
13,Phasi Charoen,3279,0,Gym / Fitness Center,Vegetarian / Vegan Restaurant,Shopping Mall,Bakery,Women's Store,Flea Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck


#### Cluster 2

In [31]:
bangkok_merged.loc[bangkok_merged['Cluster_Labels'] == 1, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,district,housing_prices,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Nong Khaem,5574,1,Thai Restaurant,Noodle House,Chinese Restaurant,Flea Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
18,Lat Krabang,2131,1,Thai Restaurant,Restaurant,Café,Chinese Restaurant,Asian Restaurant,Shipping Store,Gym / Fitness Center,Badminton Court,Food Court,Food
26,Chom Thong,3279,1,Thai Restaurant,Noodle House,Floating Market,Gay Bar,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
27,Thawi Watthana,5902,1,Thai Restaurant,Record Shop,Asian Restaurant,Bus Station,Dog Run,Duty-free Shop,Floating Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
34,Khlong Sam Wa,2623,1,Thai Restaurant,Event Service,Noodle House,Asian Restaurant,Park,Flea Market,Floating Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
39,Bang Na,13115,1,Thai Restaurant,Convenience Store,Asian Restaurant,Resort,Sushi Restaurant,Seafood Restaurant,Chinese Restaurant,Hotel,Hotpot Restaurant,Karaoke Bar


### Cluster 3

In [32]:
bangkok_merged.loc[bangkok_merged['Cluster_Labels'] == 2, bangkok_merged.columns[[1] + list(range(5, bangkok_merged.shape[1]))]]

,district,housing_prices,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bangkok Yai,9836,2,Convenience Store,Cosmetics Shop,Ice Cream Shop,BBQ Joint,Coffee Shop,Restaurant,Fast Food Restaurant,Steakhouse,Supermarket,Asian Restaurant
1,Bangkok Noi,6557,2,Noodle House,Convenience Store,Market,Food Truck,Som Tum Restaurant,Dessert Shop,Fast Food Restaurant,Thai Restaurant,Steakhouse,Coffee Shop
3,Bang Sue,3934,2,Convenience Store,Thai Restaurant,Café,Pub,Night Market,Bar,Noodle House,Market,Train Station,Boutique
11,Phra Khanong,21311,2,Convenience Store,Noodle House,Coffee Shop,Café,Gym / Fitness Center,Ice Cream Shop,Boutique,Hotel,Flower Shop,Fried Chicken Joint
31,Bang Phlat,5902,2,Convenience Store,Thai Restaurant,Noodle House,Paper / Office Supplies Store,Market,Coffee Shop,Pharmacy,Comfort Food Restaurant,Boutique,Flower Shop
37,Sai Mai,3279,2,Seafood Restaurant,Department Store,Supermarket,Convenience Store,Fishing Spot,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
41,Prawet,5902,2,Convenience Store,Noodle House,Electronics Store,Halal Restaurant,Comfort Food Restaurant,Women's Store,Floating Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
43,Suan Luang,13115,2,Convenience Store,Café,Tea Room,Women's Store,Floating Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
46,Bang Khun Thian,4262,2,Seafood Restaurant,Convenience Store,Market,Fishing Spot,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop


# Thank you